In [1]:
import os
import openai
import json
import pandas as pd
from datasets import load_dataset

os.environ["OPENAI_API_KEY"] = ''
from openai import OpenAI
client = OpenAI()

dataset = load_dataset("facebook/flores", "all")  # Loads all languages

In [2]:
language_mapping = pd.read_csv('language_mapping.csv')
language_mapping_index = language_mapping.set_index('FLORES-200 code')['Language'].to_dict()
all_languages = list(language_mapping_index.keys())

In [3]:
flores_l = {'afr_Latn': 'Afrikaans',
 'amh_Ethi': 'Amharic',
 'arb_Arab': 'Arabic',
 'asm_Beng': 'Assamese',
 'ast_Latn': 'Asturian',
 'azj_Latn': 'Azerbaijani',
 'bel_Cyrl': 'Belarusian',
 'ben_Beng': 'Bengali',
 'bos_Latn': 'Bosnian',
 'bul_Cyrl': 'Bulgarian',
 'cat_Latn': 'Catalan',
 'ceb_Latn': 'Cebuano',
 'ces_Latn': 'Czech',
 'ckb_Arab': 'Kurdish',
 'cym_Latn': 'Welsh',
 'dan_Latn': 'Danish',
 'deu_Latn': 'German',
 'ell_Grek': 'Greek',
 'eng_Latn': 'English',
 'est_Latn': 'Estonian',
 'fin_Latn': 'Finnish',
 'fra_Latn': 'French',
 'gaz_Latn': 'Oromo',
 'gle_Latn': 'Irish',
 'glg_Latn': 'Galician',
 'guj_Gujr': 'Gujarati',
 'hau_Latn': 'Hausa',
 'heb_Hebr': 'Hebrew',
 'hin_Deva': 'Hindi',
 'hrv_Latn': 'Croatian',
 'hun_Latn': 'Hungarian',
 'hye_Armn': 'Armenian',
 'ibo_Latn': 'Igbo',
 'ind_Latn': 'Indonesian',
 'isl_Latn': 'Icelandic',
 'ita_Latn': 'Italian',
 'jav_Latn': 'Javanese',
 'jpn_Jpan': 'Japanese',
 'kam_Latn': 'Kamba',
 'kan_Knda': 'Kannada',
 'kat_Geor': 'Georgian',
 'kaz_Cyrl': 'Kazakh',
 'kea_Latn': 'Kabuverdianu',
 'khk_Cyrl': 'Mongolian',
 'khm_Khmr': 'Khmer',
 'kor_Hang': 'Korean',
 'lao_Laoo': 'Lao',
 'lin_Latn': 'Lingala',
 'lit_Latn': 'Lithuanian',
 'ltz_Latn': 'Luxembourgish',
 'lug_Latn': 'Ganda',
 'luo_Latn': 'Luo',
 'lvs_Latn': 'Latvian',
 'mal_Mlym': 'Malayalam',
 'mar_Deva': 'Marathi',
 'mkd_Cyrl': 'Macedonian',
 'mlt_Latn': 'Maltese',
 'mri_Latn': 'Maori',
 'mya_Mymr': 'Burmese',
 'nld_Latn': 'Dutch',
 'nob_Latn': 'Norwegian',
 'npi_Deva': 'Nepali',
 'nso_Latn': 'Sotho',
 'nya_Latn': 'Nyanja',
 'oci_Latn': 'Occitan',
 'ory_Orya': 'Odia',
 'pan_Guru': 'Punjabi',
 'pbt_Arab': 'Pashto',
 'pes_Arab': 'Persian',
 'pol_Latn': 'Polish',
 'por_Latn': 'Portuguese',
 'ron_Latn': 'Romanian',
 'rus_Cyrl': 'Russian',
 'slk_Latn': 'Slovak',
 'slv_Latn': 'Slovenian',
 'smo_Latn': 'Samoan',
 'sna_Latn': 'Shona',
 'snd_Arab': 'Sindhi',
 'som_Latn': 'Somali',
 'spa_Latn': 'Spanish',
 'srp_Cyrl': 'Serbian',
 'swe_Latn': 'Swedish',
 'swh_Latn': 'Swahili',
 'tam_Taml': 'Tamil',
 'tel_Telu': 'Telugu',
 'tgk_Cyrl': 'Tajik',
 'tgl_Latn': 'Tagalog',
 'tha_Thai': 'Thai',
 'tur_Latn': 'Turkish',
 'ukr_Cyrl': 'Ukrainian',
 'umb_Latn': 'Umbundu',
 'urd_Arab': 'Urdu',
 'uzn_Latn': 'Uzbek',
 'vie_Latn': 'Vietnamese',
 'wol_Latn': 'Wolof',
 'xho_Latn': 'Xhosa',
 'yor_Latn': 'Yoruba',
 'zho_Hans': 'Chinese',
 'zho_Hant': 'Chinese',
 'zsm_Latn': 'Malay',
 'zul_Latn': 'Zulu'}

In [4]:
prompt_lid= "You are an assistant specialized in identifying written languages. Please identify the language in the text below. Return only the name of the language, no other text.\n Text: {}"


In [5]:
languages = list(flores_l.keys())

In [6]:
#this logic is taken from afrobench. 
import re



def get_category(prompt):
    completion = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ])
    response = completion.choices[0].message.content
    # print(response)
    return response

In [7]:
languages_to_run = languages[0:15]

In [8]:
from tqdm import tqdm
result_categories = {}
result_accuracies = {}
gpt_replies = {}

for language_code in languages_to_run:
    completed = []
    if language_code in completed:
        continue
    print(language_code)
    accurate = 0
    
    result_categories[language_code]=[]
    gpt_replies[language_code] = []

    language_name = flores_l[language_code]
    
    size = 1012
    for i in tqdm(range(size)): #length of devtest
        sentence = dataset['devtest'][i]["sentence_{}".format(language_code)]
        prompt = prompt_lid.format(sentence)
        # print(prompt)
        reply = get_category(prompt)
        gpt_replies[language_code].append(reply)

        category = ""
        if language_name.lower() in reply.lower():
            category = language_name
        result_categories[language_code].append(category)
        if category == language_name:
            accurate+=1
            
    result_accuracies[language_code] = accurate

    df = pd.DataFrame({
    "gpt_answer": result_categories[language_code]
    })

    # Save to CSV
    df.to_csv("lid_predicted_gpt4o/{}.csv".format(language_code), index=False)

    df2 = pd.DataFrame({
    "gpt_reply": gpt_replies[language_code]
    })

    # Save to CSV
    df2.to_csv("lid_replies_gpt4o/{}.csv".format(language_code), index=False)
    print(accurate)
print(result_accuracies)

afr_Latn


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1012/1012 [09:13<00:00,  1.83it/s]


1012
amh_Ethi


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1012/1012 [09:05<00:00,  1.86it/s]


1012
arb_Arab


...

1010
ckb_Arab


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1012/1012 [09:39<00:00,  1.75it/s]


1012
cym_Latn


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1012/1012 [09:22<00:00,  1.80it/s]

1012
{'afr_Latn': 1012, 'amh_Ethi': 1012, 'arb_Arab': 921, 'asm_Beng': 994, 'ast_Latn': 985, 'azj_Latn': 1012, 'bel_Cyrl': 939, 'ben_Beng': 1011, 'bos_Latn': 652, 'bul_Cyrl': 1012, 'cat_Latn': 1012, 'ceb_Latn': 1011, 'ces_Latn': 1010, 'ckb_Arab': 1012, 'cym_Latn': 1012}


In [9]:
fleurs_l=[
    "afr", "amh", "ara", "hye", "asm", "ast", "azj", "bel", "ben", "bos", 
    "bul", "mya", "yue", "cat", "ceb", "hrv", "ces", "dan", "nld", "eng", 
    "est", "tgl", "fin", "fra", "ful", "glg", "lug", "kat", "deu", "ell", 
    "guj", "hau", "heb", "hin", "hun", "isl", "ibo", "ind", "gle", "ita", 
    "jpn", "jav", "kea", "kam", "kan", "kaz", "khm", "kor", "kir", "lao", 
    "lav", "lin", "lit", "luo", "ltz", "mkd", "msa", "mal", "mlt", "cmn", "mri",
    "mar", "mon", "npi", "nso", "nob", "nya", "oci", "ory", "orm", "pus",
    "fas", "pol", "por", "pan", "ron", "rus", "srp", "sna", "snd", "slk",
    "slv", "som", "ckb", "spa", "swh", "swe", "tgk", "tam", "tel", "tha",
    "tur", "ukr", "umb", "urd", "uzb", "vie", "cym", "wol", "xho", "yor", "zul"
]



In [10]:
sorted(fleurs_l)

['afr',
 'amh',
 'ara',
 'asm',
 'ast',
 'azj',
 'bel',
 'ben',
 'bos',
 'bul',
 'cat',
 'ceb',
 'ces',
 'ckb',
 'cmn',
 'cym',
 'dan',
 'deu',
 'ell',
 'eng',
 'est',
 'fas',
 'fin',
 'fra',
 'ful',
 'gle',
 'glg',
 'guj',
 'hau',
 'heb',
 'hin',
 'hrv',
 'hun',
 'hye',
 'ibo',
 'ind',
 'isl',
 'ita',
 'jav',
 'jpn',
 'kam',
 'kan',
 'kat',
 'kaz',
 'kea',
 'khm',
 'kir',
 'kor',
 'lao',
 'lav',
 'lin',
 'lit',
 'ltz',
 'lug',
 'luo',
 'mal',
 'mar',
 'mkd',
 'mlt',
 'mon',
 'mri',
 'msa',
 'mya',
 'nld',
 'nob',
 'npi',
 'nso',
 'nya',
 'oci',
 'orm',
 'ory',
 'pan',
 'pol',
 'por',
 'pus',
 'ron',
 'rus',
 'slk',
 'slv',
 'sna',
 'snd',
 'som',
 'spa',
 'srp',
 'swe',
 'swh',
 'tam',
 'tel',
 'tgk',
 'tgl',
 'tha',
 'tur',
 'ukr',
 'umb',
 'urd',
 'uzb',
 'vie',
 'wol',
 'xho',
 'yor',
 'yue',
 'zul']

In [11]:
reply = 'Chinese'
reply.lower()

'chinese'

In [12]:
'chinese' in reply.lower()

True

In [13]:
languages[90:101]

['umb_Latn',
 'urd_Arab',
 'uzn_Latn',
 'vie_Latn',
 'wol_Latn',
 'xho_Latn',
 'yor_Latn',
 'zho_Hans',
 'zho_Hant',
 'zsm_Latn',
 'zul_Latn']